In [4]:
import pandas as pd
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Mecab


In [3]:
mbti_list = ['ISTJ','ISFJ','INFJ','INTJ',
                        'ISTP','ISFP','INFP','INTP',
                        'ESTP','ESFP','ENFP','ENTP',
                        'ESTJ','ESFJ','ENFJ','ENTJ']

In [8]:
def clean_keyword(number,file_number):
    df = pd.read_csv(f'./topic_data/mbti_topic{file_number}.csv',encoding='utf-8',index_col=0)
    df.dropna(inplace=True)
    w=df[f'{mbti_list[number]}_weight']
    k=df[f'{mbti_list[number]}_keyword']
    t=pd.DataFrame([k,w]).T
    t = t.iloc[1:]
    t[f'{mbti_list[number]}_weight']=t[f'{mbti_list[number]}_weight'].astype(float)
    return t

In [9]:
def duplication_dict(df):
    key_dict = {}
    k,v=df.items()
    for i,k_ in enumerate(k[1]):
        try:
            key_dict[k_] += v[1][i]
        except KeyError:
            key_dict[k_] = v[1][i]
    df=pd.DataFrame.from_dict(key_dict,orient='index')
    return df

In [10]:
def topic_integrated(num):
    x=[clean_keyword(num,x) for x in range(1,7)]
    df=pd.DataFrame()
    for i in range(6):
        df = pd.concat([df,x[i]])
    df.index=range(len(df))
    df=duplication_dict(df)
    if num!=3:
        df=df[df[0]>0.0135]
    else:
        df=df[df[0]>0.0090]
    return df

In [11]:
topic_df = pd.DataFrame()
for i,mbti in tqdm(enumerate(mbti_list)):
    df=topic_integrated(i)
    df.columns=[f'{mbti}']
    topic_df=pd.concat([topic_df,df],axis=1)


16it [00:00, 34.33it/s]


In [12]:
topic_df

,ISTJ,ISFJ,INFJ,INTJ,ISTP,ISFP,INFP,INTP,ESTP,ESFP,ENFP,ENTP,ESTJ,ESFJ,ENFJ,ENTJ
책임감,0.052985,0.020182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
원칙,0.038691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
변화,0.044031,0.030057,NaN,NaN,NaN,NaN,NaN,NaN,0.026895,NaN,NaN,NaN,NaN,NaN,NaN,NaN
신중,0.039184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
감각,0.067246,0.039577,0.038247,0.039372,0.043562,0.08874,0.015684,0.026329,0.092526,0.101135,0.023076,0.044112,0.045819,0.070955,0.041824,0.054966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
대담,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032215
지도자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029872
성공,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047489
통솔력,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018021


In [14]:
topic_df.to_csv('final_mbti_topic2.csv',encoding='utf-8')

In [15]:
pd.read_csv('final_mbti_topic2.csv',index_col=0).dropna()

,ISTJ,ISFJ,INFJ,INTJ,ISTP,ISFP,INFP,INTP,ESTP,ESFP,ENFP,ENTP,ESTJ,ESFJ,ENFJ,ENTJ
감각,0.067246,0.039577,0.038247,0.039372,0.043562,0.088740,0.015684,0.026329,0.092526,0.101135,0.023076,0.044112,0.045819,0.070955,0.041824,0.054966
본인,0.057362,0.079425,0.088216,0.107383,0.070524,0.052217,0.096441,0.068412,0.078170,0.042844,0.033481,0.095956,0.072179,0.060382,0.079113,0.092673
능력,0.108016,0.044553,0.106780,0.105725,0.063497,0.089597,0.098156,0.149888,0.069314,0.114567,0.071245,0.104361,0.083654,0.093822,0.068821,0.084355
이해,0.096541,0.062728,0.115774,0.097217,0.089250,0.070458,0.109757,0.070075,0.088372,0.084039,0.084694,0.087893,0.093802,0.100597,0.075049,0.107544
방식,0.090858,0.047588,0.037155,0.042275,0.064782,0.074269,0.050312,0.068849,0.062577,0.050267,0.027520,0.058296,0.083047,0.045909,0.025029,0.070254
개인,0.052462,0.049512,0.071274,0.044662,0.070727,0.049531,0.052693,0.055472,0.078644,0.086601,0.067518,0.070057,0.091018,0.059086,0.046651,0.069847
사고,0.045490,0.047390,0.044703,0.111846,0.040763,0.046177,0.053398,0.095348,0.078277,0.054376,0.049585,0.071879,0.043002,0.071459,0.055745,0.091898
판단,0.033053,0.036686,0.027907,0.058674,0.043033,0.071962,0.050769,0.061402,0.056538,0.069102,0.042473,0.036749,0.078520,0.053539,0.067403,0.061950
해결,0.038413,0.034447,0.056213,0.060267,0.088577,0.082538,0.054879,0.099946,0.096247,0.058872,0.029247,0.072014,0.056025,0.053441,0.049246,0.062571
결정,0.036719,0.017621,0.021014,0.062652,0.041382,0.025626,0.043752,0.047052,0.067152,0.043805,0.015137,0.020409,0.051639,0.025000,0.026130,0.050294
